# Setup

In [1]:
# download red wine quality data
! mkdir wine
! wget -P wine https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

# download and unzip Musk v2
! mkdir musk
! wget -P musk https://archive.ics.uci.edu/ml/machine-learning-databases/musk/clean2.data.Z
! uncompress musk/clean2.data.Z

! git clone https://github.com/jsokolowska/anomaly-detection.git
! mv anomaly-detection ad

--2023-05-10 19:36:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘wine/winequality-red.csv’

winequality-red.csv 100%[===================>]  82.23K  --.-KB/s    in 0.05s   

2023-05-10 19:36:17 (1.47 MB/s) - ‘wine/winequality-red.csv’ saved [84199/84199]

--2023-05-10 19:36:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/musk/clean2.data.Z
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1470557 (1.4M) [application/x-httpd-php]
Saving to: ‘musk/clean2.data.Z’

clean2.data.Z       100%[=============

In [2]:
# install pyod - not present by default in google colab
! pip install pyod


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184113 sha256=9b050e7301eb8eae12cd7f7d8d135dfc0bc82608ce340fb4730bd44e7997a413
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [3]:
WINE_PATH = "./wine/winequality-red.csv"
MUSK_PATH = "./musk/clean2.data"

In [4]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

def score( y_true, y_pred, y_proba):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1, roc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Parameter Tuning

In [5]:
from sklearn.model_selection import GridSearchCV
from ad.src.anomaly_detection import ClusterBasedAnomalyDetection
from sklearn.cluster import Birch
from sklearn.model_selection import StratifiedKFold
from ad.src.data_loading import load_wine, load_musk
import pandas as pd

In [6]:
df = pd.DataFrame(index=["musk", "wine"], columns=["birch-ldcof", "birch-cblof", "kmeans-ldcof", "kmeans-cblof"])
# Narrowed down params grid for quick execution
birch_grid = {
    "alpha": [0.8, 0.9, 0.95],
    "beta": [3, 10],
    "contamination": [0.1],
    "n_clusters" : [ 5, 10],
    "threshold": [0.2, 0.5, 0.7]
}
kmeans_grid = {
    "alpha": [0.8, 0.9, 0.95],
    "beta": [3],
    "contamination": [0.1],
    "n_clusters": [ 5, 10],
    "random_state": [4512],
    "n_init": [10]
}
N_SPLITS = 3
X_train, X_test, y_train, y_test  = load_wine(WINE_PATH)
skf = StratifiedKFold(n_splits=N_SPLITS, random_state = 91, shuffle = True)

Zbiór hiperparametrów i ilość podzbiorów w sprawdzianie krzyżowym zostały zawężone w celu zmniejszenia czasu wykonania się notatnika.

In [7]:
from sklearn.cluster import KMeans
clustering_algs = [("kmeans", KMeans(), kmeans_grid), ("birch", Birch(), birch_grid)]
measures = ["ldcof", "cblof"]
datasets = {"musk": load_musk(MUSK_PATH), "wine": load_wine(WINE_PATH)}

for dataset_name, values in datasets.items():
    X_train, X_test, y_train, y_test  = values
    for name, algorithm, param_grid in clustering_algs:
        for measure in measures:
            cbad = ClusterBasedAnomalyDetection(clustering_estimator=algorithm, dissimilarity_measure=measure)

            search = GridSearchCV(param_grid=param_grid, estimator=cbad, scoring="roc_auc", cv = 5, n_jobs=1)
            search.fit(X_train, y_train)
            print(f"[{name}/{measure}/{dataset_name}] Best params: {search.best_params_}")
            print(f"[{name}/{measure}/{dataset_name}] Best score: {search.best_score_}")
            df[f"{name}-{measure}"][dataset_name] = search.best_params_


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
324 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 84, in fit
    self._chosen_measure.fit(X)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 156, in fit
    raise e
  File "/kaggle/working/ad/src/anomaly_detection.py", line 147, in fit
    self._set_big_clusters()
  File "/kaggle/working/ad/s

[kmeans/ldcof/musk] Best params: {'alpha': 0.8, 'beta': 3, 'contamination': 0.1, 'n_clusters': 5, 'n_init': 10, 'random_state': 4512}
[kmeans/ldcof/musk] Best score: 0.731697522336632


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
324 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 84, in fit
    self._chosen_measure.fit(X)
  File "/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py", line 194, in fit
    self._set_small_large_clusters(n_samples)
  File "/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py", line 2

[kmeans/cblof/musk] Best params: {'alpha': 0.95, 'beta': 3, 'contamination': 0.1, 'n_clusters': 10, 'n_init': 10, 'random_state': 4512}
[kmeans/cblof/musk] Best score: 0.4181319438290222


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
828 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
828 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 84, in fit
    self._chosen_measure.fit(X)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 156, in fit
    raise e
  File "/kaggle/working/ad/src/anomaly_detection.py", line 147, in fit
    self._set_big_clusters()
  File "/kaggle/working/ad/

...

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
432 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 84, in fit
    self._chosen_measure.fit(X)
  File "/kaggle/working/ad/src/anomaly_detection.py", line 156, in fit
    raise e
  File "/kaggle/working/ad/src/anomaly_detection.py", line 147, in fit
    self._set_big_clusters()
  File "/kaggle/working/ad/

[birch/ldcof/wine] Best params: {'alpha': 0.8, 'beta': 3, 'contamination': 0.1, 'n_clusters': 10, 'threshold': 0.7}
[birch/ldcof/wine] Best score: 0.5317538118145919


/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"
/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"
/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"
/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The

[birch/cblof/wine] Best params: {'alpha': 0.9, 'beta': 3, 'contamination': 0.1, 'n_clusters': 5, 'threshold': 0.2}
[birch/cblof/wine] Best score: 0.458695405039281


/opt/conda/lib/python3.10/site-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
432 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/kaggle/working/ad/src/anomaly_detection.py

In [9]:
df.head()

,birch-ldcof,birch-cblof,kmeans-ldcof,kmeans-cblof
musk,"{'alpha': 0.8, 'beta': 10, 'contamination': 0....","{'alpha': 0.95, 'beta': 3, 'contamination': 0....","{'alpha': 0.8, 'beta': 3, 'contamination': 0.1...","{'alpha': 0.95, 'beta': 3, 'contamination': 0...."
wine,"{'alpha': 0.8, 'beta': 3, 'contamination': 0.1...","{'alpha': 0.9, 'beta': 3, 'contamination': 0.1...","{'alpha': 0.8, 'beta': 3, 'contamination': 0.1...","{'alpha': 0.9, 'beta': 3, 'contamination': 0.1..."


In [10]:
df.to_csv("./tuning-results")